In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

In [2]:
def process_image(img):
    """Resize, redice and expand image.

    Args:
        img (): original image.
    
    Returns:
        image: ndarray(64, 64, 3): processed image.
    """
    
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, 0)  # Add batch dimension.
    
    return image

In [3]:
def get_classes(file_path):
    """Get classes from file.
    
    Args:
        file_path (str): path to classes file.
    
    Returns:
        classes (list): list of classes.
    """
    with open(file_path) as f:
        classes = f.readlines()
    classes = [c.strip() for c in classes]
    
    return classes

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxed on the image.

    Args:
        image: original image.
        boxes: boxes of objects.
        scores: scores of objects.
        classes: classes of objects.
        all_classes: list of all classes.
    """

    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box
        
        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
        
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        mess = '%s: %.3f' % (all_classes[cl], score)
        cv2.putText(image, mess, (top, left - 12),
                    0, 1e-3 * image.shape[0], (0, 255, 0), 2)


In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect objects in image.

    Args:
        image: original image.
        yolo: yolo v3 model.
        all_classes: list of all classes.
    
    Returns:
        image: image with boxes.
    """
    
    pimage = process_image(image)
    
    start = time.time()
    boxes, scores, classes = yolo.predict(pimage, image.shape)
    end = time.time()
    
    print("Time: {:.2f}s, Detect Objects: {:d}.".format(end - start, len(boxes)))
    
    if len(boxes) != 0:
        draw(image, boxes, scores, classes, all_classes)
    
    return image

In [6]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [9]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

In [10]:
f = 'jingxiang-gao-489454-unsplash.jpg'
path = 'images/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


## See Lesson Notebooks